In [1]:
import pandas as pd
from pathlib import Path

# 📂 Path to your panel dataset
INPUT_PANEL = Path(r"D:\McGill\Final Course Images\FINAL COURSE STUFF\building_month_fire_panel_feat_eng.csv")
OUTPUT_PANEL = Path(r"D:\McGill\Final Course Images\FINAL COURSE STUFF\future_fire_risk_panel_6m.csv")

# 🧮 Load the panel
panel_df = pd.read_csv(INPUT_PANEL, parse_dates=["month"])

# ✅ Get the latest month in your data
last_month = panel_df["month"].max()
print(f"Last month in dataset: {last_month.date()}")

# 🏢 Unique buildings
building_ids = panel_df["ID_UEV"].unique()

# 📆 Create future months (6 months after last month)
future_months = pd.date_range(start=last_month + pd.offsets.MonthBegin(), periods=6, freq="MS")

# 📋 Cartesian product of buildings × months
future_panel = pd.MultiIndex.from_product(
    [building_ids, future_months],
    names=["ID_UEV", "month"]
).to_frame(index=False)

# 🔁 Merge static features (last known for each building)
static_columns = [
    "MUNICIPALITE", "ETAGE_HORS_SOL", "NOMBRE_LOGEMENT", "AGE_BATIMENT",
    "CODE_UTILISATION", "CATEGORIE_UEF", "SUPERFICIE_TERRAIN", "SUPERFICIE_BATIMENT",
    "NO_ARROND_ILE_CUM", "RATIO_SURFACE", "DENSITE_LOGEMENT", "HAS_MULTIPLE_LOGEMENTS",
    "FIRE_FREQUENCY_ZONE", "FIRE_RATE_ZONE", "FIRE_COUNT_LAST_YEAR_ZONE",
    "BUILDING_COUNT", "FIRE_RATE_ZONE_NORM", "FIRE_COUNT_LAST_YEAR_ZONE_NORM"
]

# Get last values per building
latest_static = (
    panel_df.sort_values("month")
    .groupby("ID_UEV")[static_columns]
    .last()
    .reset_index()
)

# ➕ Merge into future panel
future_panel = future_panel.merge(latest_static, on="ID_UEV", how="left")

# 🕓 Add time features
future_panel["month_num"] = future_panel["month"].dt.month
future_panel["year"] = future_panel["month"].dt.year

# 🔧 Add zero placeholders for rolling features
rolling_cols = [
    "fire_last_1m", "fire_last_2m", "fire_last_3m",
    "fire_cumcount", "fire_rolling_3m", "fire_rolling_6m", "fire_rolling_12m"
]
for col in rolling_cols:
    future_panel[col] = 0

# 💾 Save
future_panel.to_csv(OUTPUT_PANEL, index=False)
print(f"✅ Future 6-month panel saved to:\n{OUTPUT_PANEL}")

Last month in dataset: 2025-05-01
✅ Future 6-month panel saved to:
D:\McGill\Final Course Images\FINAL COURSE STUFF\future_fire_risk_panel_6m.csv
